In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf as sql

ModuleNotFoundError: No module named 'pandasql'

In [ ]:
df = pd.read_csv('../data/final.csv')
df_cops = pd.read_csv('../data/2010_police_num_by_county.csv')
df_people = pd.read_csv('../data/people.csv')

In [ ]:
pd.set_option('display.max_columns', 35)

In [ ]:
df.shape

In [ ]:
df_cops.shape

In [ ]:
df_people.shape

In [ ]:
df = pd.merge(df, df_cops, how='left', left_on='foreign_key', right_on='foreign_police')
df.shape

In [ ]:
df = df.dropna()

In [ ]:
df = pd.merge(df, df_people, how='left', left_on='fips', right_on='FIPS_people')
df.shape

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
df.head(2)

In [ ]:
df = df.drop(columns=['fips', 'foreign_key', 'state_police', 'county_police', 'foreign_police', 'FIPS_people'])

In [ ]:
j=[]
for i in df['population_total_2010']:
    j.append(i.replace(',', ''))
df['population_total_2010'] = j
df['population_total_2010']

k=[]
for m in df['number_police']:
    k.append(m.replace(',', ''))
df['number_police'] = k

In [ ]:
cols = ['avg_ann_pay_per_emp_large', 'avg_ann_pay_per_emp_sme', 'avg_ann_pay_per_emp_total']
for i in cols:
    df[i] = [txt if txt != '#DIV/0!' else np.nan for txt in df[i]]

In [ ]:
col_names = list(df.columns)
col_names = [i for i in col_names if not i in ['state', 'county_name']]

for b in col_names:
    try:
        df[b] = df[b].astype(float)
    except:
        pass

In [ ]:
df = df.dropna(subset=['avg_ann_pay_per_emp_sme', 'avg_ann_pay_per_emp_total'])

In [ ]:
df.shape

In [ ]:
df = df.fillna(0)

In [ ]:
per_capita_police = df['number_police'] / df['population_total_2010']

In [ ]:
police_1000 = per_capita_police * 1000

In [ ]:
df = df.rename(columns={'number_police': 'police_per_1000'})

In [ ]:
df['police_per_1000'] = police_1000

In [ ]:
df.to_csv('../data/final_work_data.csv', index=False)